In [1]:
import sqlite3 
import pandas as pd 
import requests
import json  
from datetime import datetime
import secrets
import telegram #pip install python-telegram-bot --upgrade
import time
import math

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 500) 
pd.set_option('display.max_rows', 500) 

def connect_create_table(name_db='db.db'):
    conn = sqlite3.connect(name_db)  # You can create a new database by changing the name within the quotes
    c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

    # Create table - CLIENTS
    c.execute('''CREATE TABLE if not exists POSITIONS
                 ([id] INTEGER PRIMARY KEY,
                 [date] DATETIME,
                 [encryptedUid] TEXT,
                 [nick] TEXT,
                 [symbol] TEXT,
                 [status] TEXT,
                 [entryPrice] FLOAT,
                 [markPrice] FLOAT,
                 [pnl] FLOAT,
                 [roe] FLOAT,
                 [amount] FLOAT,
                 [updateTimeStamp] TEXT,
                 [request_id] INT
                 )''')
    conn.commit()

connect_create_table()

In [2]:
def insert_in_table(trader_info, request_id, vector, status):
    encryptedUid = trader_info[0]
    nick         = trader_info[1]
    
    try:
        sqliteConnection = sqlite3.connect('db.db')
        cursor = sqliteConnection.cursor()
        
        symbol      = vector[0]
        entryPrice  = vector[1]
        markPrice   = vector[2]
        pnl         = vector[3]
        row         = vector[4]
        try: amount = float(vector[5])
        except: amount = 0
        updateTimestamp = time.time()
        date        = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Need to add more info of student (such as the txns)
        sqlite_insert_with_param = """INSERT INTO POSITIONS 
                          (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, roe, amount, updateTimeStamp, request_id) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?);"""

        data_tuple = (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, row, amount, updateTimestamp,request_id) 
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()

In [3]:
def do_query(db="db.db", query="SELECT * FROM POSITIONS"):    
    con = sqlite3.connect(db)
    df = pd.read_sql_query(query, con)
    
    return df

In [4]:
def get_positions(trader):
    endpoint = "https://www.binance.com/bapi/futures/v1/public/future/leaderboard/getOtherPosition"
    params   =  {"encryptedUid":trader,  "tradeType":"PERPETUAL"}
    headers  =  {"content-type":"application/json;charset=UTF-8"}
    response = requests.post(endpoint, json=params, headers=headers) #Add proxies = proxyDict # Attention to JSON = PARAMS
    result   = response.json()
    
    return result

In [5]:
def send_message(text, chat_id = "-1001455341077"):
    my_bot  = telegram.Bot("814569733:AAGmMk_P4p8EdLkj-fj2fVziyByxHIcJpm0")
    my_bot.send_message(chat_id=chat_id, text=text, parse_mode = "Markdown")

In [6]:
def send_message_discord(text):
    #Webhook of my channel. Click on edit channel --> Webhooks --> Creates webhook
    webhook = "https://discord.com/api/webhooks/892399337731133500/_IE9zchjLp3lRrImBt8sp_YgrI8A9Kan1Vc6wv7eY7vvkVNEAGF-YkVkiUF4pKxT5J3R"
    data    = {"content": text}
    response = requests.post(webhook, json=data)

In [13]:
def create_message(trader, row, status):
    trade_type = "NONE"
    
    if row["amount_new"] > 0:
        side = "long"
        if row["difference"] > 0:
            trade_type = "BUY"
        else:
            trade_type = "SELL"

    if row["amount_new"] < 0:
        side = "short"
        if row["difference"] > 0:
            trade_type = "BUY"                        
        else:
            trade_type = "SELL" 
    
    current_position = str(round(row["amount_new"],4))
    old_position     = str(round(row["amount_old"],4))
    price            = str(round(row["markPrice_new"],4))
    difference       = str(abs(row["difference"]))
    symbol           = row["symbol"]
    date             = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
       
    message =  "💰" + trader + " " + date + "\nPosition change from " + old_position + symbol + " to " + current_position + symbol + ".\n" + trade_type + " " + difference + symbol + " at " + price + "$" + "\nStatus:" + status 
    return message 

In [ ]:
names    = {"CCF3E0CB0AAD54D9D6B4CEC5E3E741D2":"TraderT", 
            "49A7275656A7ABF56830126ACC619FEB":"ADnefina", 
            "A532C4316C00206168F795EDFBB3E164":"ClickHereNow", 
            "D3AFE978B3F0CD58489BC27B35906769":"CheerWindCard",
            "FB23E1A8B7E2944FAAEC6219BBDF8243":"BananaMan1", 
            "F318AE1B1BB3AEF4EDBD36E5AE250CD1":"BigPiePlayer", 
            "2F9D01E80429F97670415A5A5DDD9405":"CryptoPinoy",
            "95ACEF9493428B28CCADDA38A2127859":"ChurchChalkUnder",
            "382066A3597798ED9F69FECBCABAA00D":"911korn",
            "15A34DAA83C4B2D7F3A4BBAB4D2FECCA":"BakeAlsoShare",
            "3E478F9133714216B608E95B4ACF1FA3":"SABSAB",
            "B936AEC059917C5A730D45922A9F4334":"GeboyMujaer",
            "8D27A8FA0C0A726CF01A7D11E0095577":"Nothingness",
            "C6EC5B8B2D08F5748A1D8D3D849023A3":"GetSwifty",
            "191792DA7B3EEB10DAF1D51B6926F2E1":"Mikkiko",}

#names = {"A532C4316C00206168F795EDFBB3E164":"ClickHereNow"}

status = "OPEN"
error_download = False
traders  = list(names.keys())
keys     = ["symbol", "entryPrice", "markPrice", "pnl", "roe", "amount"]

while True:
    for trader in traders:
        print(names[trader])
        trader_info = [trader, names[trader]]    # Write the name of the trader
        
        # Download data
        try:
            position_array   = get_positions(trader)     # Get the current position of the trader
            positions        = position_array["data"]["otherPositionRetList"]
            count = do_query(query="SELECT COUNT(*) as count FROM positions")["count"][0]
            if count > 0:
                request_id  = int(do_query(query="select MAX(id) as id from positions")["id"][0])
            else: 
                request_id = 0

        except:
            print(f"Couldn't download position for {names[trader]}")
            continue #This is useful not to mix stuff between positions and no positions        
        
        # Insert in the database what we downloaded
        if not positions:
            print(f"No positions for {names[trader]}, but still save.")
            empty    = [0,0,0,0,0,0] # Need to get the timestamp right!
            insert_in_table(trader_info, request_id, empty, "CLOSE")

        if positions:
            for i in range(len(positions)):
                row = pd.DataFrame(positions).loc[i]
                vector = [row["symbol"], row["entryPrice"], row["markPrice"], row["pnl"], row["roe"], row["amount"]]
                status = "CLOSED" if row["amount"] == 0 else "OPEN"
                insert_in_table(trader_info, request_id, vector, status)
        
        # Get last updates
        two_ids = do_query(query=f"select distinct(request_id) from POSITIONS where encryptedUid = '{trader}' order by request_id desc limit 2")
        
        if len(two_ids)>=2: # if the trader is in the database, check the delta
        
            # Get last update from trader.             
            this_id, last_id = two_ids["request_id"][0], two_ids["request_id"][1]
     
            # Get all the positions with that request_id
            new_positions   = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id = '{this_id}'")
            old_positions   = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id = '{last_id}'")

            # Compare the previous position with the new one, pair by pair
            both               = old_positions.merge(new_positions, "outer", on ="symbol", suffixes=('_old', '_new')).fillna(0) #Filling nas (if previous or new position is na, is because is not present, hence 0)
            both["difference"] = round(both["amount_new"] - both["amount_old"],4)
            print(both[["symbol","id_old","id_new","amount_old","amount_new","difference"]])

            for i in range(len(both)):
                row = both.loc[i]
                if row["status_old"] == "OPEN":
                    if row["difference"] != 0:   
                        if   names[trader] == "TraderT":      chat_id = -542063577
                        elif names[trader] == "ClickHereNow": chat_id = -591308509
                        else:   chat_id = -530891051    
                        message = create_message(names[trader], row, status)

                        try: 
                            send_message(message, chat_id)
                            print(message)
                        except: print("Too many messages")
                        if names[trader] in ["TraderT","ADnefina"] : send_message_discord(message)
                        time.sleep(2)

            time.sleep(2)

TraderT
   request_id
0         903
1         901
     symbol  id_old  id_new  amount_old  amount_new  difference
0   OMGUSDT     902     904    13094.00    13094.00        0.00
1  DYDXUSDT     903     905    22545.70    22545.70        0.00
ADnefina
   request_id
0         905
1         855
    symbol  id_old  id_new  amount_old  amount_new  difference
0  BTCUSDT     856     906      163.19      163.19        0.00
ClickHereNow
   request_id
0         906
1         856
          symbol  id_old  id_new  amount_old  amount_new  difference
0        BTCBUSD     857     907       24.66       24.66        0.00
1        ADAUSDT     858     908   -46268.00   -46268.00        0.00
2        BNBUSDT     859     909     -323.07     -323.07        0.00
3        ETCUSDT     860     910    -1232.84    -1232.84        0.00
4        ETHUSDT     861     911     -363.40     -350.10       13.30
5       DOGEUSDT     862     912  -416000.00  -416000.00        0.00
6        TRXUSDT     863     913   -41325.0

In [13]:
do_query(query="select MAX(id) as id from positions")["id"][0]

112

In [11]:
do_query(query=f"select distinct(request_id) from POSITIONS where encryptedUid = '{trader}' order by request_id desc limit 2")

,request_id
0,420
1,392


In [14]:
do_query(query="select*from positions where nick='ClickHereNow'")

,id,date,encryptedUid,nick,symbol,status,entryPrice,markPrice,pnl,roe,amount,updateTimeStamp,request_id
0,1,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,BTCBUSD,OPEN,49094.41,47539.10,-38361.61,-0.33,24.66,1633111346.06114,0
1,2,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,ADAUSDT,OPEN,2.23,2.23,-277.41,-0.04,-39415.00,1633111346.08116,0
2,3,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,BNBUSDT,OPEN,409.70,416.73,-2273.44,-0.30,-323.07,1633111346.09615,0
3,4,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,ETCUSDT,OPEN,51.17,51.05,146.11,0.02,-1232.84,1633111346.11565,0
4,5,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,ETHUSDT,OPEN,3244.83,3268.09,-8443.06,-0.07,-363.00,1633111346.13002,0
5,6,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,DOGEUSDT,OPEN,0.21,0.22,-2603.65,-0.58,-416000.00,1633111346.14398,0
6,7,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,TRXUSDT,OPEN,0.09,0.09,-5.12,-0.00,-69358.00,1633111346.15775,0
7,8,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,RLCUSDT,OPEN,3.87,3.75,-10656.82,-0.17,84955.80,1633111346.17176,0
8,9,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,XLMUSDT,OPEN,0.30,0.30,-61.97,-0.01,-93988.00,1633111346.18379,0
9,10,2021-10-01 20:02:26,A532C4316C00206168F795EDFBB3E164,ClickHereNow,ADABUSD,OPEN,2.23,2.23,395.79,0.07,48523.00,1633111346.19828,0
